In [2]:
import time
import numpy as np
import tensorflow as tf

加载数据

In [3]:
with open ('anna.txt','r') as f:
    text=f.read()

构建字符集合

In [4]:
vocab=set(text)#无重复集合
vocab

{'\n',
 ' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

字符-数字映射字典

In [5]:
vocab_to_int={c:i for i ,c in enumerate(vocab)}
vocab_to_int

{'\n': 15,
 ' ': 0,
 '!': 40,
 '"': 33,
 '$': 46,
 '%': 76,
 '&': 79,
 "'": 59,
 '(': 3,
 ')': 35,
 '*': 27,
 ',': 64,
 '-': 18,
 '.': 71,
 '/': 7,
 '0': 16,
 '1': 38,
 '2': 75,
 '3': 6,
 '4': 25,
 '5': 5,
 '6': 43,
 '7': 14,
 '8': 82,
 '9': 68,
 ':': 48,
 ';': 80,
 '?': 21,
 '@': 51,
 'A': 74,
 'B': 4,
 'C': 70,
 'D': 57,
 'E': 12,
 'F': 37,
 'G': 56,
 'H': 62,
 'I': 49,
 'J': 11,
 'K': 29,
 'L': 34,
 'M': 45,
 'N': 78,
 'O': 28,
 'P': 65,
 'Q': 30,
 'R': 55,
 'S': 22,
 'T': 1,
 'U': 67,
 'V': 26,
 'W': 53,
 'X': 32,
 'Y': 66,
 'Z': 41,
 '_': 44,
 '`': 42,
 'a': 54,
 'b': 69,
 'c': 9,
 'd': 63,
 'e': 24,
 'f': 77,
 'g': 8,
 'h': 81,
 'i': 39,
 'j': 58,
 'k': 60,
 'l': 31,
 'm': 17,
 'n': 52,
 'o': 73,
 'p': 50,
 'q': 2,
 'r': 72,
 's': 19,
 't': 61,
 'u': 47,
 'v': 36,
 'w': 23,
 'x': 10,
 'y': 20,
 'z': 13}

数字-字符映射字典

In [6]:
int_to_vocab=dict(enumerate(vocab))
int_to_vocab

{0: ' ',
 1: 'T',
 2: 'q',
 3: '(',
 4: 'B',
 5: '5',
 6: '3',
 7: '/',
 8: 'g',
 9: 'c',
 10: 'x',
 11: 'J',
 12: 'E',
 13: 'z',
 14: '7',
 15: '\n',
 16: '0',
 17: 'm',
 18: '-',
 19: 's',
 20: 'y',
 21: '?',
 22: 'S',
 23: 'w',
 24: 'e',
 25: '4',
 26: 'V',
 27: '*',
 28: 'O',
 29: 'K',
 30: 'Q',
 31: 'l',
 32: 'X',
 33: '"',
 34: 'L',
 35: ')',
 36: 'v',
 37: 'F',
 38: '1',
 39: 'i',
 40: '!',
 41: 'Z',
 42: '`',
 43: '6',
 44: '_',
 45: 'M',
 46: '$',
 47: 'u',
 48: ':',
 49: 'I',
 50: 'p',
 51: '@',
 52: 'n',
 53: 'W',
 54: 'a',
 55: 'R',
 56: 'G',
 57: 'D',
 58: 'j',
 59: "'",
 60: 'k',
 61: 't',
 62: 'H',
 63: 'd',
 64: ',',
 65: 'P',
 66: 'Y',
 67: 'U',
 68: '9',
 69: 'b',
 70: 'C',
 71: '.',
 72: 'r',
 73: 'o',
 74: 'A',
 75: '2',
 76: '%',
 77: 'f',
 78: 'N',
 79: '&',
 80: ';',
 81: 'h',
 82: '8'}

对文本进行转码

In [7]:
encoded=np.array([vocab_to_int[c] for c in text],dtype=np.int32)
print(len(encoded))
encoded

1985223


array([70, 81, 54, ..., 19, 71, 15])

In [8]:
def get_batches(arr,n_seqs,n_steps):#10 50
    #对数据进行分割
    #arr :待分割的数组
    #n_seqs:batch中序列个数 就是前多少步作为历史输入
    #n_steps:序列长度
    batch_size=n_seqs*n_steps#N*M 10*50
    n_batches=int(len(arr)/batch_size)#1985223/50=39704
    #保留可以被分割的部分 舍弃不可分割的部分
    arr=arr[:batch_size*n_batches]# 取整数个数
    #重塑
    arr=arr.reshape((n_seqs,-1))#变为二维数组   [n_seqs,None]  10*198520
    for n in range(0,arr.shape[1],n_steps):#(0,198520,50)
        x=arr[:,n:n+n_steps]#x是10*50的二维数组
        y=np.zeros_like(x)
        #y比x会向后错位一个字符
        y[:,:-1],y[:,-1]=x[:,1:],y[:,0]
        yield x,y

In [9]:
batches=get_batches(encoded,10,50)
x,y=next(batches)
print('x=',x.shape)
print('y=',y.shape)

x= (10, 50)
y= (10, 50)


# 模型构建

1.输入层

In [9]:
def build_inputs(num_seqs,num_steps):#n*m的矩阵
    #num_seqs:每个batch中的序列个数  步长
    #num_steps:每个序列包含的字符数  数据长度
    inputs=tf.placeholder(tf.int32,shape=(num_seqs,num_steps),name='inputs')
    targets=tf.placeholder(tf.int32,shape=(num_seqs,num_steps),name='targets')
    keep_prob=tf.placeholder(tf.float32,name='keep_prob')
    return inputs,targets,keep_prob

2.LSTM层

In [10]:
def build_lstm(lstm_size,num_layers,batch_size,keep_prob):
    #lstm_size:隐藏节点数
    #num_layers:lstm层的数目
    #batch_size:num_seqs * num_steps
    #keep_prob
    #构建lstm的基本单元
    #lstm=tf.nn.run_cell.BasicLSTMCell(lstm_size)#找不到该函数
    #drop=tf.nn.rnn_cell.DropoutWrapper(lstm,output_keep_prob=keep_prob)
    def build_cell(lstm_size,keep_prob):
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    #堆叠多层
    #cell=tf.contrib.rnn.MultiRNNCell([drop for _ in range(num_layers)])
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    initial_state=cell.zero_state(batch_size,tf.float32)
    return cell ,initial_state

3.输出层   
每个字符经过LSTM后的输出大小是1xL(隐藏层的节点数量) N x M x L
输出要与全连接层建立连接
重塑为(N x M) x L  
整个LSTM层到softmax层的大小为L x Vocab_size  就是字符合集

In [11]:
def build_output(lstm_output,in_size,out_size):
    #输出结果按照列concate,例如[[1,2,3],[7,8,9]],
    #tf.concat的结果是[1,2,3,7,8,9]
    #np.shape((lstm_output)=（64,50,128）
    seq_output=tf.concat(lstm_output,1)
    #重塑
    x=tf.reshape(seq_output,[-1,in_size])
    
    #连接LSTM输入到softmax layer
    with tf.variable_scope('softmax'):
        softmax_w=tf.Variable(tf.truncated_normal([in_size,out_size],stddev=0.1))
        softmax_b=tf.Variable(tf.zeros(out_size))
    
    logits=tf.matmul(x,softmax_w)+softmax_b
    #概率
    out=tf.nn.softmax(logits,name='predictions')
    return out,logits

4.训练计算误差

In [12]:
def build_loss(logits,targets,lstm_size,num_classes):
    #logits  全连接层输出结果 没有经过softmax
    #targets 目标字符
    #lstm_size  cell隐藏层节点数
    #num_classes vocab_size
    
    #对target进行编码
    y_one_hot=tf.one_hot(targets,num_classes)
    y_reshaped=tf.reshape(y_one_hot,logits.get_shape())
    
    loss=tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=y_reshaped)
    loss=tf.reduce_mean(loss)
    
    return loss

5.optimizer
lstm 解决梯度弥散问题  即累加
gradient clippling 的方式来防止梯度爆炸。
即通过设置一个阈值，当 gradients 超过这个阈值时，就将它重置为阈值大小，这就保证了梯度不会变得很大

In [13]:
def build_optimizer(loss,learning_rate,grad_clip):
    tvars=tf.trainable_variables()
    #tf.clip_by_global_norm会返回 clip 以后的 gradients 以及 global_norm
    grads,_=tf.clip_by_global_norm(tf.gradients(loss,tvars),grad_clip)
    train_op=tf.train.AdamOptimizer(learning_rate)
    optimizer=train_op.apply_gradients(zip(grads,tvars))
    return optimizer

6.模型组合

In [14]:
class charRNN:
    def __init__(self,num_classes,batch_size=64,num_steps=50,
                      lstm_size=128,num_layers=2,learning_rate=0.001,
                      grad_clip=5,sampling=False):
        #采用SGD
        if sampling==True:
            batch_size,num_steps=1,1
        else:
            batch_size,num_steps=batch_size,num_steps
        tf.reset_default_graph()
        
        #输入
        self.inputs,self.targets,self.keep_prob=build_inputs(batch_size,num_steps)
        
        #lstm
        cell,self.initial_state=build_lstm(lstm_size,num_layers,batch_size,self.keep_prob)
        
        #编码
        x_one_hot=tf.one_hot(self.inputs,num_classes)
        
        #运行RNN
        outputs,state=tf.nn.dynamic_rnn(cell,x_one_hot,initial_state=self.initial_state)
        self.final_state=state
        
        #预测结果
        self.prediction,self.logits=build_output(outputs,lstm_size,num_classes)
        
        #loss 和 optimizer
        self.loss=build_loss(self.logits,self.targets,lstm_size,num_classes)
        self.optimizer=build_optimizer(self.loss,learning_rate,grad_clip)
        

In [15]:
batch_size=100
num_steps=100
lstm_size=512
num_layers=2
learning_rate=0.001
keep_prob=0.5

In [16]:
epochs=20
#每n轮进行一次变量保存
save_every_n=200

model=charRNN(len(vocab),batch_size=batch_size,num_steps=num_steps,
              lstm_size=lstm_size,num_layers=num_layers,
              learning_rate=learning_rate)

saver=tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    count=0
    for e in range(epochs):
        new_state=sess.run(model.initial_state)
        loss=0
        for x,y in get_batches(encoded,batch_size,num_steps):
            count+=1
            start=time.time()
            feed={model.inputs:x,
                  model.targets:y,
                  model.keep_prob:keep_prob,
                  model.initial_state:new_state}
            
            batch_loss,new_state,_=sess.run([model.loss,
                                             model.final_state,
                                             model.optimizer],
                                             feed_dict=feed)
            
            end=time.time()
            if count%10==0:
                print('轮数：{}/{}...'.format(e+1,epochs),
                      '训练步数：{}...'.format(count),
                      '训练误差：{:.4f}...'.format(batch_loss),
                      '{:,.4f} sec/batch'.format((end-start)))
            if (count % save_every_n==0):
                saver.save(sess,'checkpoints/i{}.ckpt'.format(count,lstm_size))
    saver.save(sess,'checkpoints/i{}.ckpt'.format(count,lstm_size))
            

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



KeyboardInterrupt: 